In [1]:
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, models, transforms
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import torch
from glob import glob
import xarray as xr
import rasterio as rio
import rioxarray
import math

In [2]:
# define model
def conv3x3(in_channels, out_channels, stride=1, padding=1, bias=True):
    return nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=padding, bias=bias)


def conv1x1(in_channels, out_channels):
    return nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1)


def check_valid_activation(choice):
    if choice not in ['relu', 'lrelu', 'prelu']:
        raise ValueError(f"'{choice}' is not a valid activation function. Choose among ['relu', 'lrelu', 'prelu'].\n")


def upconv(in_channels, out_channels, mode='transpose'):
    # stride=2 implies upsampling by a factor of 2
    get_up_mode = nn.ModuleDict([
        ['bilinear', nn.Sequential(nn.Upsample(mode='bilinear', scale_factor=2), conv1x1(in_channels, out_channels))],
        ['transpose', nn.ConvTranspose2d(in_channels, out_channels, kernel_size=2, stride=2)]
    ])

    return get_up_mode[mode]


def get_activation(choice):
    activation_functions = nn.ModuleDict([
        ['relu', nn.ReLU(inplace=True)],
        ['lrelu', nn.LeakyReLU(inplace=True)],
        ['prelu', nn.PReLU()]
        ])
    return activation_functions[choice]


def conv_block(in_channels, out_channels, activation='relu', do_BN=True, *args, **kwargs):
    """
    Partial encoder block consisting of a 3×3 convolutional layer with stride 1, followed by batch normalization
    (optional) and a non-linear activation function.
    """

    if do_BN:
        return nn.Sequential(
            conv3x3(in_channels, out_channels, bias=False, *args, **kwargs),
            nn.BatchNorm2d(out_channels),
            get_activation(activation)
        )
    else:
        return nn.Sequential(
            conv3x3(in_channels, out_channels, bias=True, *args, **kwargs),
            get_activation(activation)
        )


def conv_up_block(in_channels, out_channels, activation='relu', do_BN=True, up_mode='transpose', *args, **kwargs):
    """
    Decoder block consisting of an up-convolutional layer, followed by a 3×3 convolutional layer with stride 1,
    batch normalization (optional), and a non-linear activation function.
    """

    if do_BN:
        return nn.Sequential(
            upconv(in_channels, in_channels, up_mode),
            nn.Sequential(
                conv3x3(in_channels, out_channels, bias=False, *args, **kwargs),
                nn.BatchNorm2d(out_channels),
                get_activation(activation))
            )
    else:
        return nn.Sequential(
            upconv(in_channels, in_channels, up_mode),
            nn.Sequential(
                conv3x3(in_channels, out_channels, bias=True, *args, **kwargs),
                get_activation(activation))
            )


def bottleneck(in_channels, out_channels, activation='relu', do_BN=True, *args, **kwargs):
    """
    Bottleneck block.
    """

    if do_BN:
        return nn.Sequential(
            conv3x3(in_channels, out_channels, bias=False, *args, **kwargs),
            nn.BatchNorm2d(out_channels),
            get_activation(activation)
        )
    else:
        return nn.Sequential(
            conv3x3(in_channels, out_channels, bias=True, *args, **kwargs),
            get_activation(activation)
        )


class SkipConnection(nn.Module):
    def __init__(self):
        super(SkipConnection, self).__init__()

    def forward(self, x_skip, x_up):
        return x_skip + x_up


class UNet(nn.Module):
    def __init__(self, n_input_channels=2, start_kernel=64, max_filter_depth=512, depth=5,
                 act_fn_encoder='relu', act_fn_decoder='relu', act_fn_bottleneck='relu', up_mode='transpose',
                 do_BN=False, bias_conv_layer=True, outer_skip=True, outer_skip_BN=False):
        """
        UNet network architecture.
        :param n_input_channels:    int, number of input channels
        :param start_kernel:        int, number of filters of the first convolutional layer in the encoder
        :param max_filter_depth:    int, maximum filter depth
        :param depth:               int, number of downsampling and upsampling layers (i.e., number of blocks in the
                                    encoder and decoder)
        :param act_fn_encoder:      str, activation function used in the encoder
        :param act_fn_decoder:      str, activation function used in the decoder
        :param act_fn_bottleneck:   str, activation function used in the bottleneck
        :param up_mode:             str, upsampling mode
        :param do_BN:               boolean, True to perform batch normalization after every convolutional layer,
                                    False otherwise
        :param bias_conv_layer:     boolean, True to activate the learnable bias of the convolutional layers,
                                    False otherwise
        :param outer_skip:          boolean, True to activate the long residual skip connection that adds the
                                    initial DSM to the output of the last decoder layer, False otherwise
        :param outer_skip_BN:       boolean, True to add batch normalization to the long residual skip connection,
                                    False otherwise
        """

        super(UNet, self).__init__()

        check_valid_activation(act_fn_encoder)
        check_valid_activation(act_fn_decoder)
        check_valid_activation(act_fn_bottleneck)

        if up_mode not in ['transpose', 'bilinear']:
            raise ValueError(f"'{up_mode}' is not a valid mode for upsampling. Choose among ['transpose', 'bilinear'] "
                             "to specify 'up_mode'.\n")

        self.n_input_channels = n_input_channels
        self.start_kernel = start_kernel
        self.depth = depth
        self.act_fn_encoder = act_fn_encoder
        self.act_fn_decoder = act_fn_decoder
        self.act_fn_bottleneck = act_fn_bottleneck
        self.up_mode = up_mode
        self.max_filter_depth = max_filter_depth
        self.do_BN = do_BN
        self.bias_conv_layer = bias_conv_layer
        self.do_outer_skip = outer_skip
        self.do_outer_skip_BN = outer_skip_BN
        self.filter_depths = [self.start_kernel * (2 ** i) for i in range(self.depth)]

        # Restrict the maximum filter depth to a predefined value
        self.filter_depths = [self.max_filter_depth if i > self.max_filter_depth else i for i in self.filter_depths]

        # Set up the encoder
        self.encoder = nn.ModuleList()
        self.encoder.append(nn.Sequential(
            conv_block(self.n_input_channels, self.start_kernel, activation=self.act_fn_encoder, do_BN=self.do_BN),
            nn.MaxPool2d(kernel_size=2, stride=2)
            ))

        for in_channel, out_channel in zip(self.filter_depths, self.filter_depths[1:]):
            self.encoder.append(nn.Sequential(
                conv_block(in_channel, out_channel, activation=self.act_fn_encoder, do_BN=self.do_BN),
                nn.MaxPool2d(kernel_size=2, stride=2)
            ))

        # Set up the bottleneck
        self.bottleneck = bottleneck(self.filter_depths[-1], self.filter_depths[-1], activation=self.act_fn_bottleneck,
                                     do_BN=self.do_BN)

        # Set up the decoder
        self.decoder = nn.ModuleList()
        self.filter_depths_up = list(reversed(self.filter_depths))

        for in_channel, out_channel in zip(self.filter_depths_up[:-1], self.filter_depths_up[1:]):
            self.decoder.append(conv_up_block(in_channel, out_channel, activation=self.act_fn_decoder,
                                              up_mode=self.up_mode, do_BN=self.do_BN))
        self.decoder.append(upconv(self.filter_depths_up[-1], self.filter_depths_up[-1], up_mode))

        # Set up the final layer of the decoder
        self.last_layer = conv3x3(self.start_kernel, 1, bias=self.bias_conv_layer)

        # Skip connection
        self.skipconnect = SkipConnection()

        # Batch normalization added to the long residual skip connection
        if self.do_outer_skip:
            self.layer_outer_skip = nn.ModuleList()
            if self.do_outer_skip_BN:
                self.layer_outer_skip.append(nn.BatchNorm2d(1))
            self.layer_outer_skip.append(SkipConnection())

    def forward(self, x, dem):
        skip_connections = []
        x = torch.cat((x, dem), dim=1)
        out = x

        # Encoder (save intermediate outputs for skip connections)
        for index, layer in enumerate(self.encoder):
            layer_conv = layer[:-1]  # all layers before the pooling layer (at depth index)
            layer_pool = layer[-1]   # pooling layer (at depth index)

            out_before_pool = layer_conv(out)
            skip_connections.append(out_before_pool)
            out = layer_pool(out_before_pool)

        # Bottleneck
        out = self.bottleneck(out)

        # Decoder + skip connections
        index_max = len(self.decoder) - 1
        for index, layer in enumerate(self.decoder):
            if index <= index_max - 1:
                layer_upconv = layer[0]  # upconv layer
                layer_conv = layer[1::]  # all other layers (conv, batchnorm, activation)

                out_temp = layer_upconv(out)
                out = self.skipconnect(skip_connections[-1 - index], out_temp)
                out = layer_conv(out)
            else:
                out_temp = layer(out)   # upconv of last layer
                out = self.skipconnect(skip_connections[-1 - index], out_temp)

        # Last layer of the decoder
        out = self.last_layer(out)

        # Add long residual skip connection
        if self.do_outer_skip:
            if self.layer_outer_skip.__len__() == 2:
                # pipe input through a batch normalization layer before adding it to the output of the last
                # decoder layer
                bn = self.layer_outer_skip[0]
                x_0 = x[:, 0, :, :]       # use channel 0 only
                x_0 = x_0.unsqueeze(1)
                x = bn(x_0)

            # add (batchnorm) input to the output of the last decoder layer
            add = self.layer_outer_skip[-1]
            x_0 = x[:, 0, :, :]
            x_0 = x_0.unsqueeze(1)

            out = add(x_0, out)  # use channel 0 only

        return out

In [3]:
#load model
model = UNet()
model.load_state_dict(torch.load('../..//noisemodel1.4_174epochs'))
model.eval()

UNet(
  (encoder): ModuleList(
    (0): Sequential(
      (0): Sequential(
        (0): Conv2d(2, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): ReLU(inplace=True)
      )
      (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (1): Sequential(
      (0): Sequential(
        (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): ReLU(inplace=True)
      )
      (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (2): Sequential(
      (0): Sequential(
        (0): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): ReLU(inplace=True)
      )
      (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (3): Sequential(
      (0): Sequential(
        (0): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): ReLU(inplace=True)
      )
      (1): MaxPool2d(kernel_size=2, s

In [4]:
# load in single igram and other data 
def hyp3_to_xarray_single(path):
    '''
    Reads unwrapped phase, coherence, and DEM into xarray dataset from single hyp3 folder 
    '''
    # globs for data to load
    unw_phase_path = glob(f'{path}/*unw_phase.tif')[0]
    dem_path = glob(f'{path}/*dem.tif')[0]

    # list granules for coordinate
    granule = os.path.split(unw_phase_path)[-1][0:-14]

    # read unw_phase into data array and assign coordinates
    da = xr.open_dataset(unw_phase_path)
    da = da.assign_coords({'granule':('granule', [granule])})
    
    # concatenate into dataset and rename variable
    ds = da.rename({'band_data': 'unw_phase'})

    #open coherence and dem into datasets
    dem_ds = xr.open_dataset(dem_path)

    # add coherence and dem to unw_phase dataset
    ds['elevation'] = (('band', 'y', 'x'), dem_ds.band_data.values)

    # remove band coordinate
    ds = ds.squeeze()

    return ds

In [5]:
# function to prepare arrays for model run
def arrays_to_tensor(ds, norm=True, norm_list = [-41, 41, -366, 4173]):
    
    unw_phase_ds = ds.unw_phase.interpolate_na(dim='x', use_coordinate=False)
    unw_phase_ds = unw_phase_ds.interpolate_na(dim='y', use_coordinate=False)
    
    test_tensor = torch.Tensor(unw_phase_ds.to_numpy()).nan_to_num(0)
    dem_tensor = torch.Tensor(ds.elevation.to_numpy()).nan_to_num(0)
    
    if norm==True:
        test_tensor = 2*(((test_tensor-norm_list[0])/(norm_list[1]-norm_list[0])))-1
        dem_tensor = 2*(((dem_tensor-norm_list[2])/(norm_list[3]-norm_list[2])))-1
    
    return test_tensor, dem_tensor


#function to return to original values
def undo_norm(array, min=-41, max=41):
    array = ((array+1)*((max-min)/2))+min
    return array

In [6]:
def tiled_prediction(ds, igram, dem, model=model, tile_size=1024):
    '''
    generate noise prediction for layer too large to run CNN
    '''
    xmin=0
    xmax=tile_size
    ymin=0
    ymax=tile_size
    
    # pad left and bottom 
    igram_pad = F.pad(igram, (0, tile_size, 0, tile_size), 'constant', 0)
    dem_pad = F.pad(dem, (0, tile_size, 0, tile_size), 'constant', 0)
    
    noise_pad = np.empty_like(dem_pad.numpy())

    for i in range(math.ceil((len(ds.x)/tile_size))):
        print(f'column {i}')
        for j in range(math.ceil((len(ds.y)/tile_size))):
            #print(f'row {j}')
            ymin = j*tile_size
            ymax = (j+1)*tile_size
            xmin = i*tile_size
            xmax = (i+1)*tile_size
            
            with torch.no_grad():
                noise = model(igram_pad[None, None, ymin:ymax, xmin:xmax], dem_pad[None, None, ymin:ymax, xmin:xmax])
                
            noise_pad[ymin:ymax, xmin:xmax] = noise.detach().squeeze().numpy()
            
    # recover original dimensions
    noise = noise_pad[0:(len(ds.y)), 0:(len(ds.x))]
    signal = igram.squeeze().numpy() - noise
    
    noise = undo_norm(noise)
    signal = undo_norm(signal)

    return noise, signal

In [7]:
def apply_cnn_correction(orbit_list, year_list, model=model):
    for orbit in orbit_list:
        for year in year_list:
            print(f'working on {orbit}, {year}')
            hyp3_path = f'/mnt/d/indennt/hyp3_app/{orbit}/{year}'
            hyp3_list = os.listdir(hyp3_path)
            for i, granule in enumerate(hyp3_list):
                granule_path = f'{hyp3_path}/{granule}'
                if os.path.exists(f'{granule_path}/{granule}_unw_phase_CNN_signal.tif'):
                    print(f'unw_phase_CNN already in {granule}, skipping') 
                    continue
                print(f'working on {granule}, {i}/{len(hyp3_list)}')
                ds = hyp3_to_xarray_single(granule_path)
                
                igram, dem = arrays_to_tensor(ds)
                
                noise, signal = tiled_prediction(ds=ds, igram=igram, dem=dem, model=model)
                
                ds['pred_noise'] = (('y', 'x'), noise)
                ds['pred_signal'] = (('y', 'x'), signal)
                
                ds.pred_noise.rio.to_raster(f'{granule_path}/{granule}_unw_phase_CNN_noise.tif')
                ds.pred_signal.rio.to_raster(f'{granule_path}/{granule}_unw_phase_CNN_signal.tif')

In [8]:
orbit_list = ['AT137']
year_list = ['2017', '2018', '2019', '2020', '2021']

In [11]:
apply_cnn_correction(orbit_list, year_list)

working on AT137, 2017
unw_phase_CNN already in S1BB_20170613T011036_20170625T011036_VVP012_INT40_G_ueF_0FFD, skipping
unw_phase_CNN already in S1BB_20170613T011036_20170707T011037_VVP024_INT40_G_ueF_BDAA, skipping
unw_phase_CNN already in S1BB_20170613T011036_20170731T011038_VVP048_INT40_G_ueF_52B1, skipping
unw_phase_CNN already in S1BB_20170613T011036_20170812T011039_VVP060_INT40_G_ueF_187A, skipping
unw_phase_CNN already in S1BB_20170613T011036_20170824T011040_VVP072_INT40_G_ueF_A6C1, skipping
unw_phase_CNN already in S1BB_20170613T011036_20170905T011040_VVP084_INT40_G_ueF_8684, skipping
unw_phase_CNN already in S1BB_20170613T011036_20170917T011041_VVP096_INT40_G_ueF_A773, skipping
unw_phase_CNN already in S1BB_20170613T011036_20170929T011041_VVP108_INT40_G_ueF_A0CA, skipping
unw_phase_CNN already in S1BB_20170613T011036_20171011T011041_VVP120_INT40_G_ueF_708A, skipping
unw_phase_CNN already in S1BB_20170625T011036_20170707T011037_VVP012_INT40_G_ueF_5390, skipping
unw_phase_CNN alr